In [133]:
from sklearn.datasets import load_boston
import numpy as np

x = load_boston().data
x = np.c_[x, np.ones(x.shape[0])]
y = load_boston().target

In [142]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=29)

x_train.shape

(379, 14)

In [145]:
def polynom(m, deg):
    n_rows, n_cols = m.shape[0], m.shape[1]
    nm = np.zeros((m.shape[0],n_cols*deg))
    for d in range(deg):
        for c in range(n_cols):
            for r in range(n_rows):
                nm[r][n_cols*d+c] = m[r][c]**(d+1)
    return nm

In [151]:
def normalEq(X, y):
    return np.linalg.inv(X.T @ X) @ X.T @ y

theta = normalEq(x_train, y_train)

print(theta)

preds = x_test @ theta

[ -9.35811334e-02   5.31800698e-02   3.67125390e-02   3.43338847e+00
  -1.48087291e+01   3.54024870e+00  -2.54930954e-03  -1.36624878e+00
   3.22244933e-01  -1.29376404e-02  -9.65171087e-01   1.10630023e-02
  -5.51213763e-01   3.59818983e+01]


In [147]:
def BGD(X, y, n_epoch, eta):
    theta = np.random.rand(X.shape[1], 1)
    m = len(X)
    for i in range(n_epoch):
        gradients = 2/m * X.T @ ((X @ theta) - y)
        theta = theta - eta * gradients
    return np.mean(theta, axis=1).reshape(X.shape[1], 1)

theta_BGD = BGD(x_train, y_train, 5000, 0.01)

print(theta_BGD)

/Users/kkgh070/anaconda3/envs/porto/lib/python3.6/site-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in subtract
  


[[ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]
 [ nan]]


In [148]:
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

polybig_features = PolynomialFeatures(degree=2, include_bias=False)
std_scaler = StandardScaler()
lin_reg = LinearRegression()
polynomial_regression = Pipeline([
        ("poly_features", polybig_features),
        ("std_scaler", std_scaler),
        ("lin_reg", lin_reg),
    ])
polynomial_regression.fit(x_train, y_train)
preds = polynomial_regression.predict(x_test)

In [149]:
m = LinearRegression()

m.fit(polynom(x_train, 2), y_train)

preds = m.predict(polynom(x_test, 2))

preds

array([ 22.20533569,  47.47341524,  24.70504269,  18.54592196,
        18.0983566 ,  17.86806448,  25.67023631,  30.8747193 ,
        15.95989866,  12.51335041,  44.47247981,  26.1824013 ,
        42.1290115 ,  12.89214364,  22.05008739,  27.64217668,
        25.58014948,  26.00179846,  19.14874769,  19.59634719,
        35.94385957,  17.88422763,  18.35132914,  18.10024263,
        31.41157596,  21.56238544,  35.83325179,  26.95474024,
        30.60848397,  33.45590308,  31.48522827,  24.79081938,
        11.95868313,  25.1152909 ,  13.79866487,   6.7920606 ,
        23.21088855,  11.30621046,  20.5316146 ,  19.07033003,
        30.6268128 ,  22.25155936,  28.28626292,  14.13491899,
        14.56772997,  16.99063137,  16.95885733,  42.64131158,
        15.33040705,  18.6367143 ,  27.11373761,  15.79761522,
        22.94361599,  17.33629323,  19.31028873,  14.12166167,
        27.91115594,  22.17521405,  21.73438445,  19.68456675,
        27.02514911,  10.48873662,  25.38784203,  21.42

In [150]:
from sklearn.metrics import explained_variance_score

explained_variance_score(y_test, preds)

0.81538313065314316